<a href="https://colab.research.google.com/github/marcelo7bastos/mba_enap_introducao_ciencia_dados/blob/main/trabalhos_realizados/Aula5_Exercicio5_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Exercício 5.1 - Solução

In [1]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


In [2]:
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = '365846072239' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df = pd.io.gbq.read_gbq('''

#####
-- ============================================================
-- Exercício 5.1
-- Objetivo: Construir um modelo unificado com População, PIB e
--           notas do IDEB (SAEB padronizadas por rede de ensino)
-- ============================================================

WITH base AS (
  -- -------------------------------------------------------------------
  -- 1) Seleção de dados de municípios, população e PIB
  -- -------------------------------------------------------------------
  SELECT
    d.id_municipio,
    d.nome AS nome_municipio,
    d.sigla_uf,
    pop.ano,
    PARSE_DATE('%Y', CAST(pop.ano AS STRING)) AS data_ano,
    pop.populacao,
    pib.pib,
    -- cálculo do PIB per capita
    ROUND(SAFE_DIVIDE(pib.pib, pop.populacao), 2) AS pib_per_capita
  FROM `basedosdados.br_bd_diretorios_brasil.municipio` AS d

  -- JOIN com população (2002 a 2018)
  LEFT JOIN `basedosdados.br_ibge_populacao.municipio` AS pop
    ON d.id_municipio = pop.id_municipio
   AND pop.ano BETWEEN 2002 AND 2018

  -- JOIN com PIB (2002 a 2018)
  LEFT JOIN `basedosdados.br_ibge_pib.municipio` AS pib
    ON d.id_municipio = pib.id_municipio
   AND pop.ano = pib.ano
   AND pop.ano BETWEEN 2002 AND 2018
),

ideb_pivot AS (
  -- -------------------------------------------------------------------
  -- 2) Seleção e pivotagem dos dados de IDEB
  --    - Filtra apenas redes de interesse (municipal, estadual, federal, pública)
  --    - Elimina registros sem nota (NULL)
  --    - PIVOT transforma valores da coluna 'rede' em colunas distintas
  -- -------------------------------------------------------------------
  SELECT
    id_municipio,
    ano,
    `municipal` AS nota_saeb_municipal,
    `estadual`  AS nota_saeb_estadual,
    `federal`   AS nota_saeb_federal,
    `publica`   AS nota_saeb_publica
  FROM (
    SELECT
      id_municipio,
      ano,
      LOWER(rede) AS rede,
      nota_saeb_media_padronizada
    FROM `basedosdados.br_inep_ideb.municipio`
    WHERE LOWER(rede) IN ('municipal','estadual','federal','publica')
      AND nota_saeb_media_padronizada IS NOT NULL   -- <<< remove os NULLs já aqui
  )
  PIVOT (
    ANY_VALUE(nota_saeb_media_padronizada)
    FOR rede IN ('municipal','estadual','federal','publica')
  )
)

-- -------------------------------------------------------------------
-- 3) Integração final: Junta a base populacional/PIB com as notas do IDEB
-- -------------------------------------------------------------------
SELECT
  b.id_municipio,
  b.nome_municipio,
  b.sigla_uf,
  b.ano,
  b.data_ano,
  b.populacao,
  b.pib,
  b.pib_per_capita,
  i.nota_saeb_municipal,
  i.nota_saeb_estadual,
  i.nota_saeb_federal,
  i.nota_saeb_publica
FROM base AS b
LEFT JOIN ideb_pivot AS i
  ON b.id_municipio = i.id_municipio
 AND b.ano = i.ano
ORDER BY b.sigla_uf, b.nome_municipio, b.ano;

#####

''', project_id=project_id)

df.head()

/tmp/ipython-input-1545595636.py:6: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df = pd.io.gbq.read_gbq('''


,id_municipio,nome_municipio,sigla_uf,ano,data_ano,populacao,pib,pib_per_capita,nota_saeb_municipal,nota_saeb_estadual,nota_saeb_federal,nota_saeb_publica
0,1200013,Acrelândia,AC,2002,2002-01-01,8454,39622000,4686.78,NaN,NaN,NaN,NaN
1,1200013,Acrelândia,AC,2003,2003-01-01,8695,49216000,5660.26,NaN,NaN,NaN,NaN
2,1200013,Acrelândia,AC,2004,2004-01-01,10668,59175000,5546.96,NaN,NaN,NaN,NaN
3,1200013,Acrelândia,AC,2005,2005-01-01,11451,64688000,5649.11,4.168093,3.972965,NaN,3.882167
4,1200013,Acrelândia,AC,2006,2006-01-01,11786,70531000,5984.30,NaN,NaN,NaN,NaN


In [12]:
df

,id_municipio,nome_municipio,sigla_uf,ano,data_ano,populacao,pib,pib_per_capita,nota_saeb_municipal,nota_saeb_estadual,nota_saeb_federal,nota_saeb_publica
0,1200013,Acrelândia,AC,2002,2002-01-01,8454,39622000,4686.78,NaN,NaN,NaN,NaN
1,1200013,Acrelândia,AC,2003,2003-01-01,8695,49216000,5660.26,NaN,NaN,NaN,NaN
2,1200013,Acrelândia,AC,2004,2004-01-01,10668,59175000,5546.96,NaN,NaN,NaN,NaN
3,1200013,Acrelândia,AC,2005,2005-01-01,11451,64688000,5649.11,4.168093,3.972965,NaN,3.882167
4,1200013,Acrelândia,AC,2006,2006-01-01,11786,70531000,5984.30,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
94620,1722107,Xambioá,TO,2014,2014-01-01,11722,318805000,27197.15,NaN,NaN,NaN,NaN
94621,1722107,Xambioá,TO,2015,2015-01-01,11709,313481000,26772.65,5.101155,5.016527,NaN,NaN
94622,1722107,Xambioá,TO,2016,2016-01-01,11695,296638000,25364.51,NaN,NaN,NaN,NaN
94623,1722107,Xambioá,TO,2017,2017-01-01,11683,341493000,29229.91,5.047833,5.174167,NaN,5.159500


In [3]:
## Explique o que fez na sua query e por que.
"""
1. Parti do exercício 3.1
   - Eu já tinha a junção de População + PIB por município e ano.
   - Isso me dava as métricas de base: população, PIB e PIB per capita.

2. Incluí o IDEB (exercício 5.1)
   - O enunciado pedia para acrescentar a tabela `br_inep_ideb.municipio`.
   - Eu trouxe a coluna `nota_saeb_media_padronizada` para as quatro redes:
     Municipal, Estadual, Federal e Pública.

3. Transformei linhas em colunas (PIVOT)
   - A tabela do IDEB tinha uma linha por rede.
   - Para não multiplicar registros, usei o operador PIVOT do BigQuery.
   - Assim, cada município/ano ficou em uma única linha, com 4 colunas:
     nota_saeb_municipal, nota_saeb_estadual, nota_saeb_federal, nota_saeb_publica.

4. Limpei os dados nulos
   - Filtrei `nota_saeb_media_padronizada IS NOT NULL` antes do PIVOT.
   - Dessa forma, evitei colunas ou linhas cheias de valores nulos.

5. Juntei as bases
   - Fiz um LEFT JOIN final entre a base de População+PIB e a base IDEB pivotada.
   - Usei como chave (id_municipio, ano).
   - Com o LEFT JOIN, garanti que não perderia municípios/anos sem nota no IDEB.

Resumo:
→ Peguei o que já existia (População + PIB) e acrescentei as notas do IDEB,
organizadas em colunas por rede. O resultado é uma base mais completa, que
permite analisar economia e educação juntas no mesmo painel.
""";


## [Demonstração](https://drive.google.com/file/d/1PYtx7LSVGYzv3pcjPY6VB8uO7P1Cl0cF/view?usp=sharing) de como criar um dataset

## Submeta o seu dataframe criando uma tabela no BigQuery

In [4]:
df.to_gbq("enapcd2021.pibpercapita",
              project_id=project_id,
              chunksize=40000,
              if_exists='replace',
              )

/tmp/ipython-input-1875309816.py:1: FutureWarning: to_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.to_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.to_gbq
  df.to_gbq("enapcd2021.pibpercapita",
100%|██████████| 1/1 [00:00<00:00, 6204.59it/s]


# Exercício 5.2 - Solução

## Trazendo os dados do exercício anterior

In [7]:
##
import pandas as pd
## Defina o id do seu projeto no bigquery!!!!!
project_id = '365846072239' # Defina o id do seu projeto no bigquery!!!!!
## Defina o id do seu projeto no bigquery!!!!!

df_31 = pd.io.gbq.read_gbq('''
                        SELECT
                          d.id_municipio,
                          d.nome AS nome_municipio,
                          d.sigla_uf,
                          pop.ano,
                          PARSE_DATE('%Y', CAST(pop.ano AS STRING)) AS data_ano,
                          pop.populacao,
                          pib.pib,
                          ROUND(SAFE_DIVIDE(pib.pib, pop.populacao), 2) AS pib_per_capita
                        FROM `basedosdados.br_bd_diretorios_brasil.municipio` AS d

                        -- junta com população (se não tiver população, fica NULL)
                        LEFT JOIN `basedosdados.br_ibge_populacao.municipio` AS pop
                          ON d.id_municipio = pop.id_municipio
                          AND pop.ano BETWEEN 2002 AND 2018

                        -- junta com pib (se não tiver pib, fica NULL)
                        LEFT JOIN `basedosdados.br_ibge_pib.municipio` AS pib
                          ON d.id_municipio = pib.id_municipio
                        AND pop.ano = pib.ano
                        AND pop.ano BETWEEN 2002 AND 2018

                        ;''',
                        project_id=project_id)

df_31.head()

/tmp/ipython-input-3309524099.py:7: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_31 = pd.io.gbq.read_gbq('''


,id_municipio,nome_municipio,sigla_uf,ano,data_ano,populacao,pib,pib_per_capita
0,1100809,Candeias do Jamari,RO,2002,2002-01-01,14314,61407000,4290.00
1,1100809,Candeias do Jamari,RO,2003,2003-01-01,14874,71276000,4791.99
2,1100809,Candeias do Jamari,RO,2004,2004-01-01,17338,93368000,5385.17
3,1100809,Candeias do Jamari,RO,2005,2005-01-01,16700,102722000,6151.02
4,1100809,Candeias do Jamari,RO,2006,2006-01-01,17346,110797000,6387.47


## Criando o dataframe com notas do IDEB

In [10]:
sql = """
-- IDEB por rede pivotado em colunas (Municipal, Estadual, Federal, Pública)
SELECT
  id_municipio,
  ano,
  municipal AS nota_saeb_municipal,
  estadual  AS nota_saeb_estadual,
  federal   AS nota_saeb_federal,
  publica   AS nota_saeb_publica
FROM (
  SELECT
    id_municipio,
    ano,
    LOWER(rede) AS rede,              -- normaliza rótulo da rede
    nota_saeb_media_padronizada
  FROM `basedosdados.br_inep_ideb.municipio`
  WHERE LOWER(rede) IN ('municipal','estadual','federal','publica')
    AND nota_saeb_media_padronizada IS NOT NULL
)
PIVOT (
  ANY_VALUE(nota_saeb_media_padronizada)
  FOR rede IN ('municipal','estadual','federal','publica')
)
;  -- fim da query
"""

df_ideb = pd.io.gbq.read_gbq(
    sql,
    project_id=project_id,
    location="US"  # recomendado para datasets do basedosdados
)

df_ideb.head()

/tmp/ipython-input-3554625053.py:32: FutureWarning: read_gbq is deprecated and will be removed in a future version. Please use pandas_gbq.read_gbq instead: https://pandas-gbq.readthedocs.io/en/latest/api.html#pandas_gbq.read_gbq
  df_ideb = pd.io.gbq.read_gbq(


,id_municipio,ano,nota_saeb_municipal,nota_saeb_estadual,nota_saeb_federal,nota_saeb_publica
0,2931004,2009,4.100333,4.220046,NaN,4.154500
1,2909208,2009,4.759145,3.817800,NaN,3.915167
2,2903409,2009,4.015106,4.659382,NaN,3.975833
3,4122404,2009,5.946678,6.010605,NaN,5.054000
4,2916609,2009,4.687183,5.171040,NaN,4.865379


## Merge entre o dataframe do exercício 3.1 com IDEB

In [11]:
# Fazendo o merge pelo município e ano
df_merged = pd.merge(
    df_31,
    df_ideb,
    on=["id_municipio", "ano"],   # chave comum
    how="left"                    # left join -> mantém todos de df_31
)

# Conferindo resultado
print(df_merged.shape)
df_merged.head()


(94625, 12)


,id_municipio,nome_municipio,sigla_uf,ano,data_ano,populacao,pib,pib_per_capita,nota_saeb_municipal,nota_saeb_estadual,nota_saeb_federal,nota_saeb_publica
0,1100809,Candeias do Jamari,RO,2002,2002-01-01,14314,61407000,4290.00,NaN,NaN,NaN,NaN
1,1100809,Candeias do Jamari,RO,2003,2003-01-01,14874,71276000,4791.99,NaN,NaN,NaN,NaN
2,1100809,Candeias do Jamari,RO,2004,2004-01-01,17338,93368000,5385.17,NaN,NaN,NaN,NaN
3,1100809,Candeias do Jamari,RO,2005,2005-01-01,16700,102722000,6151.02,4.559167,4.410815,NaN,4.445833
4,1100809,Candeias do Jamari,RO,2006,2006-01-01,17346,110797000,6387.47,NaN,NaN,NaN,NaN
